In [1]:
!pip install -U langchain-openai -q
!pip install --upgrade langchain -q

In [1]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
from openai import OpenAI
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

Method 1: manually embed the query and query the datastore for the most relevant documents

In [3]:
chroma_client = chromadb.PersistentClient(
    path="chromadb",
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

In [4]:
#chroma_client.delete_collection(name="documents")  

collection = chroma_client.get_collection(name="documents")

In [5]:
ids_to_retrieve = ["0", "1"]
specific_data = collection.get(ids=ids_to_retrieve)#, include = ["embeddings"])
print(specific_data)

{'ids': ['0', '1'], 'embeddings': None, 'metadatas': [{'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'}, {'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'}], 'documents': ['langchain API Reference¶\nlangchain.adapters¶\nClasses¶\nadapters.openai.ChatCompletion()\nFunctions¶\nadapters.openai.aenumerate(iterable[,\xa0start])\nAsync version of enumerate.\nadapters.openai.convert_dict_to_message(_dict)\nadapters.openai.convert_message_to_dict(message)\nadapters.openai.convert_messages_for_finetuning(...)\nConvert messages to a list of lists of dictionaries for fine-tuning.\nadapters.openai.convert_openai_messages(messages)\nConvert dictionaries representing OpenAI messages to LangChain format.\nlangchain.agents¶\nAgent is a class that uses an LLM to choose a sequence of actions to take.\nIn Chains, a sequence of actions is hardcoded. In Agents,\na langu

In [6]:
all_data = collection.get()  
len(all_data)

4

In [7]:
len(all_data['ids'])

11

In [8]:
def get_embedding(text, model="text-embedding-3-small"):
    list_texts_replaced = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding
    
#get_embedding("I love Kitty")

In [9]:
query = 'How to improve the performance of LangChain?'

In [10]:
query_embedding = get_embedding(query)

In [11]:
# Use the collection's `query` method for retrieval
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5,  
)

results

{'ids': [['0', '6', '8', '3', '7']],
 'distances': [[1.1135967346221176,
   1.2127610693400703,
   1.243314445857681,
   1.315484232029164,
   1.3276152933681291]],
 'metadatas': [[{'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'},
   {'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'},
   {'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'},
   {'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'},
   {'source': 'danny-moldovan-qa-using-vector-databases-lp/apidocs/api.python.langchain.com/en/latest/api_reference.html'}]],
 'embeddings': None,
 'documents': [['langchain API Reference¶\nlangchain.adapters¶\nClasses¶\nadapters.openai.ChatCompletion()\nFunctions¶\nadapters.openai.aenumerate(iterable[,\xa0start])\nAsync ver

Method 2: By creating a retriever based on the datastore and querying it

In [33]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

# Initialize Chroma vector store
vector_store = Chroma(
    collection_name="documents",  # The name of the collection in your Chroma DB
    client=chroma_client,  # The initialized Chroma client
    embedding_function=embedding
)

In [34]:
vector_store.search('kitty bank', search_type='similarity')

[Document(page_content="callbacks.manager.CallbackManager(handlers)\nCallback manager that handles callbacks from langchain.\ncallbacks.manager.CallbackManagerForChainGroup(...)\nInitialize callback manager.\ncallbacks.manager.CallbackManagerForChainRun(*,\xa0...)\nCallback manager for chain run.\ncallbacks.manager.CallbackManagerForLLMRun(*,\xa0...)\nCallback manager for LLM run.\ncallbacks.manager.CallbackManagerForRetrieverRun(*,\xa0...)\nCallback manager for retriever run.\ncallbacks.manager.CallbackManagerForToolRun(*,\xa0...)\nCallback manager for tool run.\ncallbacks.manager.ParentRunManager(*,\xa0...[,\xa0...])\nSync Parent Run Manager.\ncallbacks.manager.RunManager(*,\xa0run_id,\xa0...)\nSync Run Manager.\ncallbacks.mlflow_callback.MlflowCallbackHandler([...])\nCallback Handler that logs metrics and artifacts to mlflow server.\ncallbacks.mlflow_callback.MlflowLogger(**kwargs)\nCallback Handler that logs metrics and artifacts to mlflow server.\ncallbacks.openai_info.OpenAICallb

In [35]:
results = vector_store.similarity_search('kitty bank', k = 3)
len(results)

3

In [36]:
results[0]

Document(page_content="callbacks.manager.CallbackManager(handlers)\nCallback manager that handles callbacks from langchain.\ncallbacks.manager.CallbackManagerForChainGroup(...)\nInitialize callback manager.\ncallbacks.manager.CallbackManagerForChainRun(*,\xa0...)\nCallback manager for chain run.\ncallbacks.manager.CallbackManagerForLLMRun(*,\xa0...)\nCallback manager for LLM run.\ncallbacks.manager.CallbackManagerForRetrieverRun(*,\xa0...)\nCallback manager for retriever run.\ncallbacks.manager.CallbackManagerForToolRun(*,\xa0...)\nCallback manager for tool run.\ncallbacks.manager.ParentRunManager(*,\xa0...[,\xa0...])\nSync Parent Run Manager.\ncallbacks.manager.RunManager(*,\xa0run_id,\xa0...)\nSync Run Manager.\ncallbacks.mlflow_callback.MlflowCallbackHandler([...])\nCallback Handler that logs metrics and artifacts to mlflow server.\ncallbacks.mlflow_callback.MlflowLogger(**kwargs)\nCallback Handler that logs metrics and artifacts to mlflow server.\ncallbacks.openai_info.OpenAICallba

Method 3: By creating a QA chain using LangChain and adding a prompt template

In [37]:
retriever = vector_store.as_retriever()

In [38]:
template = """Use the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(), 
    chain_type="stuff",
    retriever=retriever, 
    return_source_documents=True,
    chain_type_kwargs={"prompt": PromptTemplate(template=template, input_variables=["context", "question"])}
)

# Ask a question
# query = "How to improve the performance of LangChain?"
query = "What is HumanRejectedException?"
result = qa_chain.invoke({"query": query})
result

{'query': 'What is HumanRejectedException?',
 'result': " HumanRejectedException is a type of exception that is thrown when a human rejects an agent's response. This can occur in a self-ask-style LLM call or in a structured chat agent.",
 'source_documents': [Document(page_content='Unified method for loading an agent from LangChainHub or local fs.\nagents.loading.load_agent_from_config(config)\nLoad agent from Config Dict.\nagents.utils.validate_tools_single_input(...)\nValidate tools for single input.\nlangchain.agents.format_scratchpad¶\nLogic for formatting intermediate steps into an agent scratchpad.\nIntermediate steps refers to the list of (AgentAction, observation) tuples\nthat result from previous iterations of the agent.\nDepending on the prompting strategy you are using, you may want to format these\ndifferently before passing them into the LLM.\nFunctions¶\nagents.format_scratchpad.log.format_log_to_str(...)\nConstruct the scratchpad that lets the agent continue its thought 

Ask directly Open AI

In [26]:
llm = OpenAI()  # Adjust temperature for creativity, e.g. temperature=0.1

In [29]:
response = llm.invoke(query)
response

"\n\nHumanRejectedException is a type of exception or error that occurs when a human user's input or action is rejected by a computer system or application. This can happen when the user's input does not match the expected format or criteria, or when the user does not have the necessary permissions or authorization to perform the requested action. HumanRejectedException is commonly used in software development to handle and communicate errors related to user interaction."